In [1]:
# Import the dependencies.
import matplotlib.pyplot as plt
import pandas as pd
import gmaps
import requests
import os
import json
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Tautira,PF,2021-03-14 22:37:07,-17.7333,-149.1500,87.80,66,20,9.22
1,1,Kruisfontein,ZA,2021-03-14 22:37:08,-34.0033,24.7314,68.00,97,100,1.99
2,2,Floro,NO,2021-03-14 22:37:09,61.5996,5.0328,39.20,56,0,3.44
3,3,Mabai,CN,2021-03-14 22:37:09,23.0128,104.4508,57.49,96,85,4.83
4,4,Ovalle,CL,2021-03-14 22:37:10,-30.5983,-71.2003,68.22,56,0,5.21


In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key = g_key)

In [5]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))


In [6]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of percent Cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Heatmap of percent wind speed
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 90


In [10]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Tautira,PF,2021-03-14 22:37:07,-17.7333,-149.1500,87.80,66,20,9.22
1,1,Kruisfontein,ZA,2021-03-14 22:37:08,-34.0033,24.7314,68.00,97,100,1.99
4,4,Ovalle,CL,2021-03-14 22:37:10,-30.5983,-71.2003,68.22,56,0,5.21
5,5,Puerto Ayora,EC,2021-03-14 22:37:10,-0.7393,-90.3518,82.40,74,40,9.22
8,8,Kapaa,US,2021-03-14 22:37:12,22.0752,-159.3190,73.40,78,75,23.02
10,10,Bredasdorp,ZA,2021-03-14 22:37:12,-34.5322,20.0403,66.20,94,99,8.05
11,11,Arraial Do Cabo,BR,2021-03-14 22:37:13,-22.9661,-42.0278,80.60,73,0,12.66
14,14,Myrtle Beach,US,2021-03-14 22:35:55,33.6891,-78.8867,64.99,72,90,5.75
15,15,Manono,CD,2021-03-14 22:37:14,-7.3000,27.4167,68.74,86,23,2.28
19,19,Tual,ID,2021-03-14 22:37:15,-5.6667,132.7500,83.53,70,92,5.23


In [11]:
preferred_cities_df.count()

City_ID       305
City          305
Country       300
Date          305
Lat           305
Lng           305
Max Temp      305
Humidity      305
Cloudiness    305
Wind Speed    305
dtype: int64

In [12]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Tautira,PF,87.80,-17.7333,-149.1500,
1,Kruisfontein,ZA,68.00,-34.0033,24.7314,
4,Ovalle,CL,68.22,-30.5983,-71.2003,
5,Puerto Ayora,EC,82.40,-0.7393,-90.3518,
8,Kapaa,US,73.40,22.0752,-159.3190,
10,Bredasdorp,ZA,66.20,-34.5322,20.0403,
11,Arraial Do Cabo,BR,80.60,-22.9661,-42.0278,
14,Myrtle Beach,US,64.99,33.6891,-78.8867,
15,Manono,CD,68.74,-7.3000,27.4167,
19,Tual,ID,83.53,-5.6667,132.7500,


In [13]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
     # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [14]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Tautira,PF,87.80,-17.7333,-149.1500,Pension Oaoa
1,Kruisfontein,ZA,68.00,-34.0033,24.7314,Oyster Bay House Rental
4,Ovalle,CL,68.22,-30.5983,-71.2003,Hotel Altos de Tuqui
5,Puerto Ayora,EC,82.40,-0.7393,-90.3518,Finch Bay Galapagos Hotel
8,Kapaa,US,73.40,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach
10,Bredasdorp,ZA,66.20,-34.5322,20.0403,Bredasdorp Country Manor
11,Arraial Do Cabo,BR,80.60,-22.9661,-42.0278,Pousada Porto Praia
14,Myrtle Beach,US,64.99,33.6891,-78.8867,Bay View Resort Myrtle Beach
15,Manono,CD,68.74,-7.3000,27.4167,Hotel hewa bora
19,Tual,ID,83.53,-5.6667,132.7500,Grand Vilia Hotel


In [15]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [16]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [17]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))